# importar librerias 

In [179]:
import numpy as np 
import pandas as pd 
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity


In [180]:
df_genres = pd.read_parquet('./data_clean_EDA/df_dummies.parquet')
df_games = pd.read_parquet('./data_clean_EDA/df_games_ML.parquet')

In [181]:
# Función para normalizar la columna 'release_date' usando StandardScaler
def normalizar_release_date(df, columna):
    scaler = StandardScaler()
    df[columna] = scaler.fit_transform(df[[columna]])
    return df

In [182]:
# Función para fusionar los DataFrames 'df_games' y 'df_genres' en 'df_merged'
def fusionar_dataframes(df1, df2, key, cols_a_incluir):
    df_merged = df1.merge(df2, on=key, how='left')
    df_final = df_merged[['id'] + cols_a_incluir]
    return df_final

In [183]:
# Función para muestrear un subconjunto de datos
def muestrear_datos(df, fraccion, estado_aleatorio):
    return df.sample(frac=fraccion, random_state=estado_aleatorio)

In [184]:
# Función para calcular la matriz de similitud del coseno
def calcular_similitud(df, features):
    matriz_similitud = cosine_similarity(df[features].fillna(0))
    return np.nan_to_num(matriz_similitud)

In [185]:
# Función para recomendar juegos basados en la similitud del coseno
def recomendacion_juego(df_sampled, similarity_matrix, game_id, top_n=5):
    ids_juegos_muestreados = df_sampled['id'].unique()
    
    if game_id not in ids_juegos_muestreados:
        return f"No se encontraron recomendaciones: {game_id} no está en los datos muestreados."
    
    indice_juego = df_sampled.index[df_sampled['id'] == game_id].tolist()
    
    if not indice_juego:
        return f"No se encontraron recomendaciones: {game_id} no está en los datos muestreados."
    
    indice_juego = indice_juego[0]
    
    puntajes_similitud = list(enumerate(similarity_matrix[indice_juego]))
    puntajes_similitud = sorted(puntajes_similitud, key=lambda x: x[1], reverse=True)

    indices_juegos_similares = [i for i, puntaje in puntajes_similitud[1:top_n+1]]
    
    nombres_juegos_similares = df_sampled['app_name'].iloc[indices_juegos_similares].tolist()
    
    mensaje_recomendacion = f"Juegos recomendados basados en el ID del juego {game_id} - {df_sampled['app_name'].iloc[indice_juego]}:"
    
    return [mensaje_recomendacion] + nombres_juegos_similares

In [186]:
# Simulación del proceso completo
def proceso_recomendacion(df_games, df_genres, game_id, top_n=5):
    # Lista de columnas a incluir
    features = ['release_date'] + list(df_genres.columns[1:])
    
    # Normalizar 'release_date' en df_games
    df_games = normalizar_release_date(df_games, 'release_date')

    # Fusionar DataFrames y seleccionar columnas
    df_merged = fusionar_dataframes(df_games, df_genres, key='id', cols_a_incluir=features)

    # Agregar columna 'app_name'
    df_final = df_merged.merge(df_games[['id', 'app_name']], on='id', how='left')

    # Muestrear los datos (20%)
    df_sampled = muestrear_datos(df_final, fraccion=0.2, estado_aleatorio=42)

    # Calcular la matriz de similitud del coseno
    similarity_matrix = calcular_similitud(df_sampled, features)

    # Obtener recomendaciones
    recomendaciones = recomendacion_juego(df_sampled, similarity_matrix, game_id, top_n)
    
    return recomendaciones

In [197]:
test_game_id = "774276"
recomendaciones = proceso_recomendacion(df_games, df_genres, test_game_id, top_n=5)

# Imprimir las recomendaciones
print(f"Recomendaciones para ID de juego {test_game_id}:")
for rec in recomendaciones:
    print(rec)

Recomendaciones para ID de juego 774276:
Juegos recomendados basados en el ID del juego 774276 - Imhotep, Pyramid Builder:
TS Marketplace: Feather River Canyon Scenario Pack 01
FSX: Steam Edition - Grumman HU-16B Albatross Add-On
TANE DLC: Newcastle Shunter
FSX Steam Edition: Endelave Airport Add-On
FSX: Steam Edition - Nordborg Airfield Add-On
